# Package import

In [1]:
from models import *
from evaluate import *
from preprocessing import *

cuda


'2.1.0+cu118'

In [19]:
# pytorch 모델링 패키지

import torch
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import torch.nn.functional as F
from torch.utils.data import  Dataset, DataLoader
import torch.optim as optim

torch.__version__

cuda


'2.1.0+cu118'

In [20]:
# 데이터 전처리 및 시각화 패키지
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler


# 평가지표
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# ETC
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings(action="ignore")

# Model

In [21]:
class LSTM_StockPrice(nn.Module):
    def __init__(self, config):
        super().__init__()
        # common
        self.dropout = nn.Dropout(config.dropout_rate)

        self.dense = nn.Linear(in_features=config.hidden_size*2, 
                                out_features=1)

        # Stock
        self.stock_lstm1 = nn.LSTM(input_size=config.stock_input, 
                             hidden_size=config.hidden_size,
                             batch_first=True)

        self.stock_lstm2 = nn.LSTM(input_size=config.hidden_size, 
                             hidden_size=config.hidden_size,
                             batch_first=True)


        
        # Index
        self.index_lstm1 = nn.LSTM(input_size=config.index_input, 
                                   hidden_size=config.hidden_size,
                                   batch_first=True)

        self.index_lstm2 = nn.LSTM(input_size=config.hidden_size, 
                                   hidden_size=config.hidden_size,
                                   batch_first=True)



    def forward(self, x_stock, x_index):
        # Stock
        stock_lstm_output, _ = self.stock_lstm1(x_stock)
        stock_drop = self.dropout(stock_lstm_output)
        stock_lstm_output, _ = self.stock_lstm2(stock_drop)
        stock_out = self.dropout(stock_lstm_output)


        # Index 
        index_lstm_output, _ = self.stock_lstm1(x_index)
        index_drop = self.dropout(index_lstm_output)
        index_lstm_output, _ = self.stock_lstm2(index_drop)
        index_out = self.dropout(index_lstm_output)


        # Concat
        comb = torch.cat((stock_out, index_out), dim=1)
        dense_output = self.dense(comb[:, -1, :])
        sigmoid_output = torch.sigmoid(dense_output)
        # predictions = self.linear(lstm_out[-1, :])  # 마지막 시퀀스 출력만 사용
        
        return sigmoid_output

# Dataset

In [44]:
class StockDataset(Dataset):
    def __init__(self, data, device, window_size=7, future_step=1):
        super().__init__()
        self.window_size = window_size
        self.future_step = future_step
        self.scaler = StandardScaler()

        X = self.scaler.fit_transform(data.drop(columns=['Date', 'Close']))
        y = data['Close'].to_numpy()[window_size+future_step-1:]
        
        X_sequencing = []
        for seq in range(len(X)-window_size):
            X_sequencing.append(X[seq:seq+window_size])

        self.X = torch.Tensor(X_sequencing).to(device)
        self.y = torch.Tensor(y).to(device)


    def get_scaler(self):
        return self.scaler
    
    def get_window_size(self):
        return self.window_size
    
    def get_future_step(self):
        return self.future_step
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [45]:
class IndexDataset(Dataset):
    def __init__(self, data, stock_target, device, window_size=7):
        super().__init__()
        self.scaler = StandardScaler()
        self.window_size = window_size

        X = self.scaler.fit_transform(data.drop(columns=['Date']))
        
        X_sequencing = []
        for seq in range(len(X)-window_size):
            X_sequencing.append(X[seq:seq+window_size])

        self.X = torch.Tensor(X_sequencing).view(X.len, 1,   ).to(device)
        self.y = stock_target.to(device)


    def get_scaler(self):
        return self.scaler
    
    def get_window_size(self):
        return self.window_size
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [23]:
# stock
AAPL = pd.read_csv('../data/AAPL.csv')

# index
NASDAQ = pd.read_csv('../data/NASDAQ.csv')
SP500 = pd.read_csv('../data/SP500.csv')
DOLLAR = pd.read_csv('../data/DOLLAR.csv')

In [41]:
window_size = 7
future_step = 1

# stock
stock_dataset = StockDataset(AAPL.drop(columns=['Close']), device=device, window_size=window_size, future_step=future_step)


# index




# DataLoader

In [ ]:
batch_size = 100

# Train

# Predcit

In [197]:
class Save_checkpoint:
    def __init__(self, score = []):
        self.score = score
    
    def check_score(self, new_score):
        self.score.append(new_score)
        if len(self.score) == 1 or len(self.score) > 1 and max(self.score) == self.score[-1]:
            return True
        else:
            return False
            
    def save_checkpoint(self, model, verbose = True):
        if verbose:
            print(f'Best Model Saved with {max(self.score)}')
            
        torch.save(model.state_dict(), 'best_model.pt')